<a href="https://colab.research.google.com/github/M4rc3l0Jr/data_upp_pipeline_data/blob/main/newdatapipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
# imports
import gspread
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import credentials
from google.auth import default, iam
from google.auth.credentials import with_scopes_if_required
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import time
import re
from unidecode import unidecode
import random
import string

warnings.filterwarnings("ignore")

data_atual = datetime.today().date()

# authentication with google
auth.authenticate_user()
creds, _ = default()
creds = with_scopes_if_required(creds, ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])

gc = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
# method to all folders from drive that we need
def get_all_folders(name):
  # listing all folders
  results = drive_service.files().list(q="mimeType='application/vnd.google-apps.folder'", pageSize=1000, fields="nextPageToken, files(id, name)").execute()
  items = results.get('files', [])

  for item in items:
    if 'tabela_de_reatailer' in item['name']:
      folder_id_retailer_table = item['id']
      file_retailer_table = drive_service.files().list(q=f"'{folder_id_retailer_table}' in parents",
                                      fields='files(name)').execute()

      # extrain file name
      file_names = [file['name'] for file in file_retailer_table.get('files', [])]
      file_names = file_names[0]
      if 'retailer' in file_names:
        data_retailer_table = gc.open(f'{file_names}').sheet1
        df_retailers = data_retailer_table.get_all_values()
        df_retailers = pd.DataFrame(df_retailers[1:], columns=df_retailers[0])

  folders = []

  # verify if folder has name we need
  for item in items:
      if name in item['name']:
        folders.append(item)

  # remove folder raw
  for item in folders:
    if name == item['name']:
      folder_id_raw = item['id']
      folders.remove(item)

  # return folder list
  return folders, folder_id_raw, df_retailers

# method to take archives of the folder
def take_archives(folder_id):
  files = drive_service.files().list(q=f"'{folder_id}' in parents",
                                  fields='files(name)').execute()

  # extrain file name
  file_names = [file['name'] for file in files.get('files', [])]

  # return file names
  return file_names

# method to read gsheets
def read_gsheets(file_names):
  df_registered = None
  df_orders = None
  for file in file_names:
    if 'cadastro' in file:
      # open file and convert in dataframe
      data_registered = gc.open(f'{file}').sheet1
      df_registered = data_registered.get_all_values()
      df_registered = pd.DataFrame(df_registered[1:], columns=df_registered[0])
      print('Tabela de clientes criada!')
    else:
      data_order = gc.open(f'{file}').sheet1
      df_orders = data_order.get_all_values()
      df_orders = pd.DataFrame(df_orders[1:], columns=df_orders[0])
      print('Tabela de pedidos criada!')

  # return df_registered and df_orders
  return df_registered, df_orders

def create_folder(folder_id, name_folder):
  parent_folder_id = folder_id

  folder_metadata = {
      'name': name_folder,
      'mimeType': 'application/vnd.google-apps.folder',
      'parents': [parent_folder_id]
  }

  new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
  folder_raw_id = new_folder.get('id')

  return folder_raw_id

def create_gsheets(folder_id, name_sheets, df):

  file_metadata = {
      'name': name_sheets,
      'parents': [folder_id],
      'mimeType': 'application/vnd.google-apps.spreadsheet'
  }
  spreadsheet = drive_service.files().create(body=file_metadata,
                                              fields='id').execute()

  sh = gc.open_by_key(spreadsheet['id'])

  # Selecionando a primeira planilha na planilha
  worksheet = sh.get_worksheet(0)

  # Convertendo o DataFrame para uma lista de listas
  data = df.values.tolist()

  # Inserindo os dados na planilha
  worksheet.clear()
  worksheet.append_row(df.columns.tolist())

  n_rows = 0

  for row in data:
      if (n_rows % 45) == 0:
        time.sleep(45)
      worksheet.append_row(row)
      n_rows += 1

def rename_columns(df):
    '''Use this method to rename all columns to transform 'Column Name' in 'column_name'
        Args:
            df: Dataframe you want rename columns
        Returns:
            df: Dataframe with raname columns
    '''
    columns = df.columns # take all df columns

    for column in columns: # loop to rename each column

        column_rename = unidecode(column).lower() # column in lowercase
        column_rename = re.sub(r'[^a-zA-Z0-9\s]', '', column_rename) # remove all special character
        column_rename = column_rename.replace(' ', '_') # change ' ' to '_'
        df.rename(columns={f'{column}':f'{column_rename}'}, inplace=True)  # rename column

    return df

def format_date(date):
    ''' Use this method to transform '%m-%d-%Y' in '%Y-%m-%d'
        Args:
            date: date column, this column need is in string format
        Returns:
            date: in format '%Y-%m-%d'
    '''
    # verify if the value is null, case is null return null
    if pd.notnull(date):
        # Check if the input is already a datetime object
        if isinstance(date, pd.Timestamp):
            return date.strftime("%Y-%m-%d")  # Format directly
        else:
            return datetime.strptime(date, "%m/%d/%Y").strftime("%Y-%m-%d")
    else:
        return None

def remove_number_address(full_address):
    '''Use this method to extract the number of the address
        Args:
            full_address: full address
        Returns:
            number: return full address number
    '''
    match = re.search(r'\d+', full_address) # to do a verify if the value has number

    if match: # case it's has return first sequence
        number = match.group()
        return number

    else: # case not, return 'sem_numero'
        number = 'sem_numero'
        return number

def remove_complement_address(full_address):
    '''Use this method to extract the complemt of the address
        Args:
            full_address: full addresss
        Returns:
            complement: return full address complement
    '''
    # Search for the first occurrence of one or more digits (\d+) in the full_address string
    match = re.search(r'\d+', full_address)

    # If a match is found:
    if match:
        # Get the index where the numeric characters end
        index_numeric = match.end()

        # Extract the part of the address string starting from the index where numeric characters end,
        # removing any leading or trailing whitespace
        complement = full_address[index_numeric:].strip()

        # Return the extracted complement part of the address
        return complement
    # If no match is found:
    else:
        # Return the original full_address string
        return full_address

def format_address(street, number, complement):
    '''Use this method to get correct address based in address data

        df[['number', 'complement', 'street']] = df.apply(lambda row: format_address(row['street'], row['number'], row['complement']), axis=1)

        Args:
            street: df['street']
            number: df['number']
            complement: df['complement']
        Return:
            street: street data
            number: number street
            complement: complement street
    '''
    # concat all values to get full address and remove possible issues when this data is inputed
    street = str(street)
    number = str(number)
    complement = str(complement)
    full_address = street + ' ' + number + ' ' + complement
    # remove address number to full_address
    number = remove_number_address(full_address)
    # recmove address complement to full_address
    complement = remove_complement_address(full_address)
    # take the name street
    street = full_address.replace(number, '').replace(complement, '')
    # return three values to three columns
    return number, complement, street

def household_type(complement):
    '''Use this method to get household type

        Args:
            complement: df['complement']
        Return:
            household_type: home, apartment or others
    '''
    # to convert input in string and lowercase
    complement = str(complement).lower()
    # if complement has 'casa' or 'cs' in complement
    if 'casa' in complement or 'cs' in complement:
        # inpout 'casa' in variable
        household = 'casa'
    # if complement has ap or is a digit:
    elif 'ap' in complement or complement.isdigit():
        # input 'apartamento' in variable
        household = 'apartamento'
    # if complement dont have 'casa' or 'apartamento'
    else:
        # input 'outros' in avariable
        household = 'outros'
    # return the household
    return household

def remove_symbol_currency(currency):
        '''Use this method to remove currency symbol
            Args:
                currency: df['Currency']
            Return:
                currency without symbol, example 'R$'
        '''
        if currency != '':
          currency = float(currency.replace('R$ ', '').replace('.', '').replace(',', '.'))
        return currency

def first_order(df_orders):
    '''Use this method to create primeira_compra column
        Args:
            df_orders: df_orders
        Return:
            df_orders with primeira_compra column
    '''
    # create auxiliar df with email and data_do_pedido column
    df_aux = df_orders[['email', 'data_do_pedido']]
    # group df by email and take retailer first order
    df_aux = df_aux.groupby('email').agg(primeira_compra=('data_do_pedido', 'min')).reset_index()
    # to do left join between df_orders and df_aux using email column
    df_orders = df_orders.merge(df_aux, how='left', on='email')
    # transform primeira_compra column in boolean value
    df_orders['primeira_compra'] = df_orders['data_do_pedido'] == df_orders['primeira_compra']
    return df_orders

def index_order(df_orders):
    '''Use this method to create index_order column
        Args:
            df_orders: df_orders
        Return:
            df_orders wiht index_order column
    '''
    # create auxiliar df with email, data_do_pedido and pedido column
    df_aux = df_orders[['email', 'data_do_pedido', 'pedido']]
    # group df by email and create a rank ascending based in data_do_pedido
    df_aux['index_order'] = df_aux.groupby('email')['data_do_pedido'].rank(method='first')
    # sort value by email and index_order
    df_aux = df_aux.sort_values(by=['email', 'index_order'])
    # drop column data_do_pedido
    df_aux.drop(columns='data_do_pedido', axis=1, inplace=True)
    # to do left join between df_orders and df_aux using email and pedido column
    df_orders = df_orders.merge(df_aux, how='left', on=['email', 'pedido'])

    return df_orders

def remove_especial_charcter(x):
  x = unidecode(x).lower()
  return x

def ticket_medio(df_orders):
    df_ticket_medio = df_orders[['email', 'pedido', 'sub_total']]
    df_ticket_medio = df_ticket_medio.groupby('email').agg(
        n_total_pedidos=('pedido', 'nunique'),
        sub_total=('sub_total', 'sum')
    ).reset_index()
    df_ticket_medio['ticket_medio'] = df_ticket_medio['sub_total']/df_ticket_medio['n_total_pedidos']
    df_ticket_medio = df_ticket_medio[['email', 'ticket_medio']]
    return df_ticket_medio

In [ ]:
def format_df_registered(df_registered):
  df_registered = rename_columns(df_registered)

  column_mapping_portuguese = {
            'street':'endereco',
            'number':'numero',
            'complement':'complemento',
            'district':'bairro',
            'city':'cidade',
            'state':'estado',
            'registered_date':'data_de_cadastro',
            'number': 'numero'
    }

  # to get all columns
  all_columns = df_registered.columns

  # loop to change name columns to portuguese
  for column in all_columns:
      # try get value based in column
      try:
          column_to_rename = column_mapping_portuguese[column]
      # case can't column_to_rename is column
      except:
          column_to_rename = column
      # rename column
      df_registered.rename(columns={
          f'{column}':f'{column_to_rename}'
      }, inplace=True)

  # loop that take only column per time:
  for column in all_columns:
      # drop all column that donst is in list
      if column not in ['email', 'endereco', 'numero', 'complemento', 'bairro', 'estado', 'cidade', 'data_de_cadastro']:
          df_registered.drop(columns=f'{column}', axis=1, inplace=True)


  # take all columns after drop the coluns that will be use
  all_columns = df_registered.columns

  # list with possible names
  columns_date = ['data']
  columns_address = ['endereco', 'numero', 'complemento']

  # list to take columns to format address
  columns_address_format = []

  # columns in right order
  right_order = ['endereco', 'numero', 'complemento']

  # loop to take  only column per time:
  for column in all_columns:

    # if column is in columns_date format date
    if any(keyword in column for keyword in columns_date):
        df_registered[column] = pd.to_datetime(df_registered[column].apply(format_date))
    # if column is in columns_address add in columns_address_format
    elif any(keyword in column for keyword in columns_address):
        columns_address_format.append(column)

  df_address = df_registered[columns_address_format]

  df_registered[['numero', 'complemento', 'endereco']] = df_address.apply(lambda row: pd.Series(format_address(row['endereco'], row['numero'], row['complemento'])), axis=1)

  df_registered['tipo_morada'] = df_registered['complemento'].apply(household_type)

  df_registered['cidade'] = df_registered['cidade'].apply(remove_especial_charcter)

  return df_registered

def format_df_order(df_orders):
    '''Use this method to restructure order dataframe
        Args:
            df_orders: dataframe with orders
        Return:
            df_orders: df_orders rename columns and only need columns
    '''
    # rename columns
    df_orders = rename_columns(df_orders)

    # get all columns
    all_columns = df_orders.columns
    # loop to drop all columns that dont go use
    for column in all_columns:
        # drop all column that donst is in list
        if column not in ['email', 'pedido', 'data_do_pedido', 'dia_do_pedido', 'quantidade_de_itens', 'código_kitprato', 'codigo_produto_simples', 'produtos', 'qtd_produtos', 'valor_unitario', 'sub_total', 'valor_descontos', 'valor_frete', 'valor_total', 'forma_de_entrega', 'forma_de_pagamento']:
          try:
            df_orders.drop(columns=f'{column}', axis=1, inplace=True)
          except:
            continue

    #create primeira_compra column
    df_orders = first_order(df_orders)

    # create index_order column
    df_orders = index_order(df_orders)

    for column in ['valor_unitario', 'sub_total', 'valor_descontos', 'valor_frete', 'valor_total']:
      df_orders[column] = df_orders[column].apply(remove_symbol_currency)

    # return dataframe adjusted
    return df_orders

def create_table_month(df_registered, df_orders):
  # format date to monday
  df_registered['data_de_cadastro'] = pd.to_datetime(df_registered['data_de_cadastro'])
  df_registered['mes'] = df_registered['data_de_cadastro'].apply(lambda x: x.strftime("%m/01/%Y"))
  df_orders['data_do_pedido'] = pd.to_datetime(df_orders['data_do_pedido'])
  df_orders['mes'] = df_orders['data_do_pedido'].apply(lambda x: x.strftime('%m/01/%Y'))
  # transform column sub_total in float
  df_orders['sub_total'] = df_orders['sub_total'].astype(float)
  df_orders['index_order'] = df_orders['index_order'].astype(int)
  # unique count per registered month
  df_cadastro_total = df_registered.groupby('mes').agg(total_cadastro=('email', 'nunique')).reset_index()
  # unique count email with ticket_medio = '' per registered_month
  df_cadastro_sem_compra = df_registered[df_registered['ticket_medio'].isna()].groupby('mes').agg(cadastro_sem_compra=('email','nunique')).reset_index()
  # avarage value first order per month
  df_avg_value_first_order = df_orders[df_orders['index_order']==1].groupby('mes').agg(avg_value_first_order=('sub_total','mean')).reset_index()
  df_avg_value_first_order['avg_value_first_order'] = df_avg_value_first_order['avg_value_first_order'].round(2)
  # avarage value to retention
  df_avg_value_retention_order = df_orders[df_orders['index_order']>1].groupby('mes').agg(total_retention=('sub_total','sum'), total_orders=('pedido','nunique')).reset_index()
  df_avg_value_retention_order['avg_value_retention'] = round(df_avg_value_retention_order['total_retention']/df_avg_value_retention_order['total_orders'],2)
  df_avg_value_retention_order.drop(columns=['total_retention', 'total_orders'], inplace=True)
  # avarage ticket_medio per month
  df_avg_ticket_medio = df_orders.groupby('mes').agg(total_retention=('sub_total','sum'), total_orders=('pedido','nunique')).reset_index()
  df_avg_ticket_medio['avg_ticket_medio'] = round(df_avg_ticket_medio['total_retention']/df_avg_ticket_medio['total_orders'],2)
  df_avg_ticket_medio.drop(columns=['total_retention', 'total_orders'], inplace=True)
  # merge all df
  df_consolidated_month = df_cadastro_total.merge(df_cadastro_sem_compra, how='left', on='mes')
  df_consolidated_month = df_consolidated_month.merge(df_avg_value_first_order, how='left', on='mes')
  df_consolidated_month = df_consolidated_month.merge(df_avg_value_retention_order, how='left', on='mes')
  df_consolidated_month = df_consolidated_month.merge(df_avg_ticket_medio, how='left', on='mes')
  df_consolidated_month['cadastro_com_compra'] = df_consolidated_month['total_cadastro'] - df_consolidated_month['cadastro_sem_compra']
  df_consolidated_month['porc_retailer_com_compra'] = round(df_consolidated_month['cadastro_com_compra']/df_consolidated_month['total_cadastro']*100,2)
  df_consolidated_month.fillna(0, inplace=True)
  new_order = ['mes', 'total_cadastro', 'cadastro_sem_compra', 'avg_value_first_order', 'avg_value_retention', 'avg_ticket_medio', 'cadastro_com_compra','porc_retailer_com_compra']
  df_consolidated_month = df_consolidated_month[new_order]

  return df_consolidated_month

def create_cohort_table(df_registered, df_orders):
  # Pegar somente a data do segundo pedido
  df_retencao_cohort = df_orders[df_orders['index_order'] == 2][['email','data_do_pedido']]
  # Pegar a data de registro
  df_registered_cohort = df_registered[['email', 'data_de_cadastro']]
  # Trunc para mês
  df_registered_cohort['cohort'] = df_registered_cohort['data_de_cadastro'].dt.strftime('%m/01/%Y')
  # Dropar a coluna data_de_cadastro
  df_registered_cohort.drop(columns='data_de_cadastro', axis=1, inplace=True)
  # Trunc para mês
  df_retencao_cohort['mes'] = df_retencao_cohort['data_do_pedido'].dt.strftime('%m/01/%Y')
  # Left join entre df_retencao_cohort e df_registered_cohort
  df_retencao_cohort = df_retencao_cohort.merge(df_registered_cohort, how='left', on='email')
  # Pegar todas as order maior ou igual a 2
  df_retencao_cohort_other = df_orders[df_orders['index_order'] >= 2][['email','data_do_pedido']]
  # Trunc para mês
  df_retencao_cohort_other['mes_other'] = df_retencao_cohort_other['data_do_pedido'].dt.strftime('%m/01/%Y')
  # Dropar a coluna data_do_pedido
  df_retencao_cohort_other.drop(columns='data_do_pedido', axis=1, inplace=True)
  # Left join entre df_retencao_cohort e df_retencao_cohort_other usando email
  df_retencao_cohort = df_retencao_cohort.merge(df_retencao_cohort_other, how='left', on='email')
  # calculando os meses
  df_retencao_cohort['mes'] = round((pd.to_datetime(df_retencao_cohort['mes_other']) - pd.to_datetime(df_retencao_cohort['mes'])).dt.days/30,0)
  # transforma em número inteiro
  df_retencao_cohort['mes'] = df_retencao_cohort['mes'].astype(int)
  # Dropa as colunas data_do_pedido e mes_other
  df_retencao_cohort.drop(columns=['data_do_pedido', 'mes_other'], axis=1, inplace=True)
  # Calcula o Total de cada cohort
  df_total_cohort = df_retencao_cohort.groupby('cohort').agg(total=('email', 'nunique')).reset_index()
  # Calcula o total por mes
  df_mes = df_retencao_cohort.groupby(['cohort', 'mes']).agg(n_recompra=('email', 'nunique')).reset_index()
  # Left join entre df_total_cohort e df_mes usando o cohort
  df_retencao_cohort = df_total_cohort.merge(df_mes, how='left', on='cohort')
  # Calcual % de recompra por cohort
  df_retencao_cohort['n_recompra'] = round(df_retencao_cohort['n_recompra']/df_retencao_cohort['total']*100,0)
  # tira os meses iniciais
  df_retencao_cohort = df_retencao_cohort[df_retencao_cohort['mes'] > 0]

  return df_retencao_cohort

def create_year_table(df_registered, df_orders):
  data_atual = datetime.today().date()
  columns = ['ano', 'tempo_retencao', 'ticket_medio']
  dados_temp = []
  df_aux = pd.DataFrame(columns=columns)

  df_take_year = list(set(list(pd.to_datetime(df_registered['data_de_cadastro']).dt.strftime('%Y'))))
  df_take_year.remove(str(data_atual.year))

  df_ano = df_registered[['email', 'data_de_cadastro']]
  # format data_de_cadastro to datetime
  df_ano['data_de_cadastro'] =  pd.to_datetime(df_ano['data_de_cadastro'])
  # take date order
  df_compra_email = df_orders[['email','data_do_pedido', 'index_order', 'pedido', 'sub_total']]
  # format data_do_petido to datetime
  df_compra_email['data_do_pedido'] = pd.to_datetime(df_compra_email['data_do_pedido'])
  df_compra_email['sub_total'] = df_compra_email['sub_total'].astype(float)
  for ano in df_take_year:
    # set start period
    periodo_inicial = f'{ano}-01-01'
    # set final period
    periodo_final = f'{ano}-12-31'
    # filter df_ano to between start and final period, take retailer that registered between this periods
    df_aux_ano = df_ano[(df_ano['data_de_cadastro'] >= periodo_inicial) & (df_ano['data_de_cadastro'] <= periodo_final)]
    # filter to take second order between start and final period
    df_compra_seg = df_compra_email[(df_compra_email['index_order']==2) & (df_compra_email['data_do_pedido'] >= periodo_inicial) & (df_compra_email['data_do_pedido'] <= periodo_final)][['email', 'data_do_pedido']]
    # filter to take last order between start and final period
    df_compra_ult = df_compra_email[(df_compra_email['data_do_pedido'] >= periodo_inicial) & (df_compra_email['data_do_pedido'] <= periodo_final)].groupby('email').agg(ult_compra=('data_do_pedido', 'max')).reset_index()
    # filter to take ticket medio between start and final period
    df_ticket_mean = df_compra_email[(df_compra_email['data_do_pedido'] >= periodo_inicial) & (df_compra_email['data_do_pedido'] <= periodo_final)].groupby('email').agg(
        n_pedidos=('pedido', 'nunique'),
        total = ('sub_total', 'sum')
    ).reset_index()
    # calculate avarage ticket
    df_ticket_mean['ticket_medio'] = df_ticket_mean['total']/df_ticket_mean['n_pedidos']
    # drop columns that will be used
    df_ticket_mean.drop(columns=['n_pedidos', 'total'], axis=1, inplace=True)
    # left join between df_compra_seg and df_compra_ult using email
    df_compra = df_compra_seg.merge(df_compra_ult, how='left', on='email')
    # left join between df_aux_ano and df_compra using email
    df_aux_ano = df_aux_ano.merge(df_compra, how='left', on='email')
    # left join between df_aux_ano and df_ticket_mean using email
    df_aux_ano = df_aux_ano.merge(df_ticket_mean, how='left', on='email')
    # set columns dias_sem_compra is final period
    df_aux_ano['dias_sem_compra'] = periodo_final
    # format dias_sem_compra to datetime
    df_aux_ano['dias_sem_compra'] = pd.to_datetime(df_aux_ano['dias_sem_compra'])
    # calculate diference between final peiod and last order
    df_aux_ano['dias_sem_compra'] = (df_aux_ano['dias_sem_compra'] - df_aux_ano['ult_compra']).dt.days
    # calculate avarage of days without order
    media_dias_sem_compra = df_aux_ano['dias_sem_compra'].mean()
    # take all retailer with dias_sem_compra less then media_dias_sem_compra
    df_aux_ano = df_aux_ano[df_aux_ano['dias_sem_compra'] < media_dias_sem_compra]
    # filter to take all retailers that are with data_do_pedido is not null
    df_aux_ano = df_aux_ano[~df_aux_ano['data_do_pedido'].isna()]
    # calculate retention months
    df_aux_ano['tempo_de_retencao'] = (df_aux_ano['ult_compra'] - df_aux_ano['data_do_pedido']).dt.days /30
    # add one in tempo_de_retencao because retailer dont do 0 month that dont purchase but yes 1 month
    df_aux_ano['tempo_de_retencao'] = df_aux_ano['tempo_de_retencao'].round(0) + 1
    # calculate avarage of the retention time
    tempo_mr = round(df_aux_ano['tempo_de_retencao'].mean(),0)
    # calculate avatage of the avarage ticket
    avarage_ticket = round(df_aux_ano['ticket_medio'].mean(),0)
    # add data in a dictionary and in list
    dados_temp.append({'ano': ano, 'tempo_retencao': tempo_mr, 'ticket_medio':avarage_ticket})
    # create dataframe this list
    df_temp = pd.DataFrame(dados_temp)
    # concat with auxiliar dataframe
    df_aux = pd.concat([df_aux, df_temp])

  # drop duplicates
  df_ltv_ano = df_aux.drop_duplicates()
  # calculate LTV
  df_ltv_ano['LTV'] = df_ltv_ano['ticket_medio']*df_ltv_ano['tempo_retencao']
  # remove all null
  df_ltv_ano = df_ltv_ano[~df_ltv_ano['LTV'].isna()]
  # format LTV
  df_ltv_ano['LTV'] = df_ltv_ano['LTV'].round(2)

  columns = ['ano', 'churn']
  dados_temp = []
  df_aux = pd.DataFrame(columns=columns)

  for ano in df_take_year:
    # set start period
    periodo_inicial = f'{ano}-01-01'
    # set final period
    periodo_final = f'{ano}-12-31'
    # filter df_ano to between start and final period, take retailer that registered between this periods
    df_cadastro_ano = df_ano[(df_ano['data_de_cadastro'] >= periodo_inicial) & (df_ano['data_de_cadastro'] <= periodo_final)]
    # total registered in year
    total_cadastrado_ano = df_cadastro_ano['email'].nunique()
    # filter order between star and final period
    df_ult_compra_ano = df_compra_email[(df_compra_email['data_do_pedido'] >= periodo_inicial) & (df_compra_email['data_do_pedido'] <= periodo_final)][['email', 'data_do_pedido']]
    # take last order
    df_ult_compra_ano = df_ult_compra_ano.groupby('email').agg(ult_compra=('data_do_pedido', 'max')).reset_index()
    # create data_atual column
    df_ult_compra_ano['data_atual'] = periodo_final
    # format column to datetime
    df_ult_compra_ano['data_atual'] = pd.to_datetime(df_ult_compra_ano['data_atual'])
    # calculate days without order
    df_ult_compra_ano['dias_sem_compra'] = (df_ult_compra_ano['data_atual'] - df_ult_compra_ano['ult_compra']).dt.days
    # calculate avarage days without order
    df_media_dias = df_ult_compra_ano[['email', 'dias_sem_compra']]
    dias_sem_compra = df_media_dias['dias_sem_compra'].mean()
    # take only retailers with days without order less then avarage days without order
    df_ult_compra_ano = df_ult_compra_ano[df_ult_compra_ano['dias_sem_compra'] < dias_sem_compra]
    # take tempo_retencao from df_ltv_ano
    df_ltv_ano_aux = df_ltv_ano[df_ltv_ano['ano'] == ano]
    tempo_medio_de_retencao = df_ltv_ano_aux['tempo_retencao'].mean()
    # format final period to datetime
    periodo_final = pd.to_datetime(periodo_final)
    # calculate time wihtout order
    df_ult_compra_ano['tempo_sem_compra'] = (periodo_final - df_ult_compra_ano['ult_compra']).dt.days/30
    # verify if client is lost
    df_ult_compra_ano['cliente_perdido'] = np.where(df_ult_compra_ano['tempo_sem_compra'] >= tempo_medio_de_retencao, True, False)
    # calculate total retailer lost
    total_perdido_ano = df_ult_compra_ano[df_ult_compra_ano['cliente_perdido'] == True]['email'].nunique()
    # calculate churn
    churn_ano = round(total_perdido_ano/total_cadastrado_ano*100,2)
    # create temp dataframe
    dados_temp.append({'ano': ano, 'churn': churn_ano})
    df_temp = pd.DataFrame(dados_temp)
    df_aux = pd.concat([df_aux, df_temp])
  # drop duplicates
  df_churn_ano = df_aux.drop_duplicates()
  # sort values by ano
  df_churn_ano.sort_values(by='ano', inplace=True)

  df_merge = df_ltv_ano.merge(df_churn_ano, how='left', on='ano')
  df_merge['coluna'] = ''
  df_merge['churn'] = df_merge['churn'].astype(str)

  return df_merge

In [ ]:
def carregando(): #function to "show" that is loading
    i = 0
    while i < 3:
        time.sleep(0.8)
        if i == 2:
          print('.')
        else:
          print('.', end='', flush=True)
        i += 1

def gerar_id_unico():
    caracteres = string.ascii_letters + string.digits  # letras maiúsculas, minúsculas e números
    id_unico = ''.join(random.choice(caracteres) for _ in range(12))
    return id_unico

def verificar_existencia(id_gerado, ids_existentes):
    return id_gerado in ids_existentes

def new_retailer(name): # main fuction to create new retailers
  name = name.lower() # take lower name

  print('\nAbrindo a pasta share', end='', flush=True)
  carregando()
  folder_id, folder_id_raw, df_retailer = get_all_folders(name) # get all folder that is in folder with name

  print('\nPegando os arquivos', end='', flush=True)
  carregando()

  if len(folder_id) > 1: # verify if this folder has more than 1 archive
    print('\nEsse e-commerce já tem as pastas padrão')
    erro = True
    return erro

  for folder in folder_id: # take folder id with share base
    if 'share' in folder['name']:
      folder_id = folder['id']

  id_retailers = list(set(list(df_retailer['id'])))

  dict_retailer = {}

  id_gerado = gerar_id_unico()
  while verificar_existencia(id_gerado, id_retailers):
    id_gerado = gerar_id_unico()
  dict_retailer['id'] = f'{id_gerado}'
  dict_retailer['nome_da_empresa'] = f'{name}'
  dict_retailer['email'] = ''
  dict_retailer['telefone'] = ''
  dict_retailer['nome'] = ''
  dict_retailer['cnpj'] = ''
  print(dict_retailer)

  erro = False

#   name_share_archives = take_archives(folder_id) # take name of the share archives

#   print('\nCriando as tabelas', end='', flush=True)
#   carregando()
#   df_registered, df_orders = read_gsheets(name_share_archives) # read gsheets and input in dataframe

#   print('\nFormatando a tabela de clientes', end='', flush=True)
#   carregando()
#   df_registered = format_df_registered(df_registered) # format df_registered

#   print('\nFormatando a tabela de pedidos', end='', flush=True)
#   carregando()
#   df_order = format_df_order(df_orders) # format df_order

#   raw_name = f'raw_{name}' # is folder name
#   print(f'\nCriando a Pasta "{raw_name}"', end='', flush=True)
#   carregando()

#   folder_raw_id = create_folder(folder_id_raw, raw_name) # take folder id
#   print('\nBaixando o arquivo da tabela de clientes', end='', flush=True)
#   carregando()

#   df_registered = df_registered.merge(ticket_medio(df_order), how='left', on='email') # calculate avarage order value and doing left join with df_registered

#   df_registered['data_de_cadastro'] = df_registered['data_de_cadastro'].apply(format_date) # fomat to datetime data_de_cadastro column
#   df_registered.to_csv('df_retialer.csv') # creating csv to df_registered

#   print('\nBaixando o arquivo da tabela de pedidos', end='', flush=True)
#   carregando()
#   df_order['data_do_pedido'] = df_order['data_do_pedido'].apply(format_date) # format to datetime data_do_pedido column
#   df_order.to_csv('df_order.csv') # creating csv to df_order

#   core_name = f'core_{name}' # is core folder name

# #### comentar o resto do código

#   print(f'\nCriando a Pasta "{core_name}"', end='', flush=True)
#   carregando()
#   folder_core_id = create_folder(folder_id_raw, core_name)
#   print(f'\nCriando a tabela "month_{name}_table_{data_atual}"', end='', flush=True)
#   carregando()
#   df_consolidated_month = create_table_month(df_registered, df_order)
#   print(f'\nCriando a tabela "year_{name}_table_{data_atual}"', end='', flush=True)
#   carregando()
#   df_year = create_year_table(df_registered, df_order)
#   print(f'\nCriando a tabela "cohort_{name}_table_{data_atual}"', end='', flush=True)
#   carregando()
#   df_cohort = create_cohort_table(df_registered, df_order)
#   print(f'\nColando os dados no google sheets', end='', flush=True)
#   carregando()
#   name_sheets = f'month_{name}_table_{data_atual}'
#   create_gsheets(folder_core_id, name_sheets, df_consolidated_month)
#   print(f'\nTabela "{name_sheets}" pronta!')
#   name_sheets = f'cohort_{name}_table_{data_atual}'
#   create_gsheets(folder_core_id, name_sheets, df_cohort)
#   print(f'\nTabela "{name_sheets}" pronta!')
#   name_sheets = f'year_{name}_table_{data_atual}'
#   create_gsheets(folder_core_id, name_sheets, df_year)
#   print(f'\nTabela "{name_sheets}" pronta!')
#   erro = False
  # return erro

In [ ]:
data_atual = str(data_atual).replace('-','_')

erro_first_answer = True
while erro_first_answer == True:
  is_new = input('É um cliente novo?\nR: ')
  if is_new.lower() in ['sim', 'yes']:
    while erro == True:
      name = input('\nArquivos de qual e-commerce você quer?\nR: ')
      if name != '':
        new_retailer(name)
  elif is_new.lower() in ['não', 'nao', 'No', 'no']:
    name = input('\nArquivos de qual e-commerce você quer?\nR:n')
    try:
        name = name.lower()
        print('\nAbrindo a pasta share', end='', flush=True)
        carregando()
        folder_id, folder_id_raw = get_all_folders(name)
        for folder in folder_id:
          if 'share' in folder['name']:
            folder_id = folder['id']
        print('\nPegando os arquivos', end='', flush=True)
        carregando()
        name_share_archives = take_archives(folder_id)
        print(name_share_archives)
        erro = False
    except:
      print('Pasta não encontrada, vamos tentar novamente?')
  else:
    print('\n Resposta aparenta não estar correta! \n')

É um cliente novo?
R: sim

Arquivos de qual e-commerce você quer?
R: teste

Abrindo a pasta share...

Pegando os arquivos...
{'id': '6JHqle7ByvTO', 'name': 'teste'}


KeyboardInterrupt: Interrupted by user